*Please install the **Matlab engine** first before importing the 'matlab.engine' library!*

*Quick guide:*
- start Matlab as administrator (to do this you might need to right click Matlab and choose "run as administrator")
- copy the following two lines to Matlab and run:

cd (fullfile(matlabroot,'extern','engines','python'))

system('python setup.py install')

*Check out more info [here](https://www.mathworks.com/help/matlab/matlab_external/install-the-matlab-engine-for-python.html)*

In [1]:
import pathlib, shutil, os, re, time, matlab.engine
import numpy as np
import pandas as pd

In [2]:
# user specified parameters

userParams = {
    'models': ['WW11'],
    'scenarios': ['s1', ], # 's3'
    'vintageDates': ['2001-02-14', ], # '2001-05-12', '2001-08-15', '2001-11-14', '2020-02-11', '2020-05-12',
    # 'engineNumbers': 1,
    # 'splitTaskBy': 'scenarios',
    'rootFolder': 'C://Users//dev//Documents//GitHub//MMB_forecast_application',
}

# assert 1 <= dictP['engineNumbers'] <= len(eval(dictP['splitTaskBy']))

mode_compute algorithms
- 4 Sims' csminwel
- 7 fminsearch
- 1 fmincon
- 3 fminunc
- 5 Marco Ratto newrat
- 6 Monte-Carlo based simulation

In [3]:
# hyper-parameters

hyperParams = {
    'chainLength': int(1e4),
    'subDraws': int(1e4/4),
    'forecastHorizon': 100,
    'chainNum': 1,
    'burnIn': 0.3,
    'scalingFactor': 0.3,
    'presample': 4,
    'mode_compute_order': [4, 4, 7, 7, 1, 1, 3, 3, 5, 5, 6],
    'nobs': 100,
}

hyperParams['sampleSize'] = hyperParams['presample'] + hyperParams['nobs']

In [4]:
paths = {'root': pathlib.Path(userParams['rootFolder']),}
paths.update({
    # 'data': pathlib.Path(paths['root'] / 'data' / 'vintage_data'),
    'models': pathlib.Path(paths['root'] / 'models'),
    'estimations': pathlib.Path(paths['root'] / 'estimations'),
    'scripts': pathlib.Path(paths['root'] / 'scripts')
})

assert np.prod([p.exists() for p in paths.values()])

os.chdir(paths['scripts'])
import generate_vintage, sample_ends_at_vintagedate

In [39]:
eng = matlab.engine.start_matlab('-desktop -nosplash')
eng.eval(f"cd('{str(paths['root'])}')", nargout=0)

In [49]:
for model in userParams['models']:
    pathModel = paths['models'] / model

    if pathModel.exists():

        modFile = model+'.mod'

        for vintageDate in userParams['vintageDates']:
            for scenario in userParams['scenarios']:
                print()
                
                # create folder and copy model files
                nameWorking = f"{model}_{vintageDate.replace('-','')}_{scenario}_{hyperParams['nobs'] - (scenario == 's1')}obs"
                pathWorking = paths['estimations'] / nameWorking
                if pathWorking.exists():
                    print(f'Warning: Folder {nameWorking} alreeady exists! Enter Y to delete it and start a new estimation. Enter C to continue without deleting anything. Enter N to stop estimation.')
                    inputValue = ''
                    while inputValue not in {'y', 'n', 'c'}:
                        inputValue = input()
                    if inputValue == 'y':
                        shutil.rmtree(pathWorking)
                        shutil.copytree(pathModel, pathWorking)
                    elif inputValue == 'c':
                        for file in pathModel.glob('*.*'):
                            shutil.copy(file, pathWorking)
                    else:
                        continue

                # retreive observed variables
                os.chdir(pathWorking)
                with open(modFile, 'r') as file:
                    script = file.read()
                lineVarobs = re.search(pattern='varobs[ _a-z0-9\n]{1,};', string=script)
                if lineVarobs == None:
                    print(f'No varobs found in {modfile}')
                    continue
                else:
                    varobs = lineVarobs.group(0).replace('varobs', ' ').replace('\n', ' ').replace(';', ' ')
                    varobs = re.sub('[ ]{1,}', ' ', varobs)
                    varobs = [inst for inst in varobs.split(' ') if len(inst)>0]

                # create and copy data
                os.chdir(paths['scripts'])
                quarterStart, quarterEnd = sample_ends_at_vintagedate.main(vintageDate=vintageDate, sampleSize=hyperParams['sampleSize'])
                generate_vintage.main(
                    vintageDate=vintageDate, quarterStart=str(quarterStart), quarterEnd=str(quarterEnd), 
                    observed=varobs, showRawTransform=False, pathExcel=pathWorking
                    )

                # write estimation command
                os.chdir(pathWorking)
                eng.eval(f"cd('{str(pathWorking)}')", nargout=0)

                xlsRange = chr(65+len(varobs)) + str(hyperParams['sampleSize'] + (scenario != 's1'))
                estimationCommand = \
                f"\n\nestimation(datafile=data_{vintageDate.replace('-','')}, xls_sheet={scenario}, xls_range=B1:{xlsRange}, presample={hyperParams['presample']}, \
                mh_replic={hyperParams['chainLength']}, mh_nblocks={hyperParams['chainNum']}, mh_drop={hyperParams['burnIn']}, mh_jscale={hyperParams['scalingFactor']}, \
                sub_draws={hyperParams['subDraws']}, forecast={hyperParams['forecastHorizon']}, \
                order=1, prefilter=0, nograph, smoother, silent_optimizer, \
                mode_compute={hyperParams['mode_compute_order'][0]}) gdp_rgd_obs;"
                script += estimationCommand

                # check whether mode compute exists
                modeFile = pathWorking / (model+'_mode.mat')
                if modeFile.exists():

                    script = script.replace(f"mode_compute={hyperParams['mode_compute_order'][0]})", f'mode_compute=0, mode_file={model}_mode)')
                    with open(modFile, 'w') as file:
                        file.write(script)

                    print('Mode file exists, will skip ML estimation ...', end=' ')
                    eng.dynare(modFile, nargout=0)
                    print(' ... succeeded.')
                    break

                # loop through all mode_compute rountines
                for i, mode_compute in enumerate(hyperParams['mode_compute_order']):
                    if i > 0:
                        script = script.replace(f"mode_compute={hyperParams['mode_compute_order'][i-1]}", f'mode_compute={mode_compute}')

                    with open(modFile, 'w') as file:
                        file.write(script)

                    try:
                        print(f'Now computing mode using algorithm {mode_compute} ...', end=' ')
                        eng.dynare(modFile, nargout=0)
                        print(' ... succeeded.', end=' ')
                        time.sleep(10)
                        eng.save('workspace', nargout=0)
                        print(' Variables saved.', end=' ')
                        time.sleep(10)
                        shutil.rmtree(pathWorking / model)
                        print(' Auxillary files deleted.')
                        break
                    except:
                        print(' ... failed. Try next.')

    else:
        print(f'Model {model} does not exist, will be skipped.')


Vintage date < End date of last observation quarter, so Last observation date = Vintage date.
Generated data with different scenarios.
Mode file exists, will skip ML estimation ... ... succeeded.


In [112]:
np.array(eng.workspace['oo_']['MeanForecast']['Mean']['gdp_rgd_obs'])

In [21]:
# eng.dynare("WW11.mod", nargout=0)
# eng.eval("options_.datafile = 'data.csv'", nargout=0)
# eng.eval("options_.mh_replic = 5000", nargout=0)
# eng.eval("options_.forecast = 5", nargout=0)
# eng.eval("options_.order = 1", nargout=0)
# eng.eval("options_.nograph = 1", nargout=0)
# eng.dynare_estimation("", nargout=0)